In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# -------------------------
# Paths & model
# -------------------------
input_path = '/home/debasish/Documents/YOLOv8/videos/bacteria'
output_base_dir = '/home/debasish/Documents/YOLOv8/outputs/bacteria_counting'
os.makedirs(output_base_dir, exist_ok=True)

image_extensions = (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff")

# -------------------------
# Create run folder (run1, run2, ...)
# -------------------------
def create_run_folder(base_dir):
    run_id = 1
    while os.path.exists(os.path.join(base_dir, f"run{run_id}")):
        run_id += 1
    run_dir = os.path.join(base_dir, f"run{run_id}")
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

run_dir = create_run_folder(output_base_dir)
print(f"[INFO] Saving outputs to {run_dir}")

# -------------------------
# Collect image files
# -------------------------
image_files = []
for root, _, files in os.walk(input_path):
    for file in files:
        if file.lower().endswith(image_extensions):
            image_files.append(os.path.join(root, file))
image_files.sort()

print(f"Total images found: {len(image_files)}")

# -------------------------
# Load YOLO model
# -------------------------
model = YOLO('/home/debasish/Documents/YOLOv8/weights/bacteria-counting/yolo12l-832.pt')

# -------------------------
# Function to count and save image
# -------------------------
def counter(image_path, save_dir):
    image = cv2.imread(image_path)
    results = model.predict(image, save=False, verbose=False, max_det=2000)

    count = 0
    for result in results:
        boxes_xywh = result.boxes.xywh.cpu().numpy()  # shape (n, 4)
        count += boxes_xywh.shape[0]

        # Draw boxes
        for box in boxes_xywh:
            x_center, y_center, w, h = box
            x1 = int(x_center - w / 2)
            y1 = int(y_center - h / 2)
            x2 = int(x_center + w / 2)
            y2 = int(y_center + h / 2)
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 1)

    cv2.putText(image, f"Total count: {count}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)

    out_name = os.path.splitext(os.path.basename(image_path))[0] + "_counted.png"
    out_path = os.path.join(save_dir, out_name)
    cv2.imwrite(out_path, image)

    return count

# -------------------------
# Run detection on all images
# -------------------------
outputs = []
for i, image_path in enumerate(image_files):
    count = counter(image_path, run_dir)
    outputs.append({"filename": os.path.basename(image_path), "count": count})
    print(f"[{i+1}/{len(image_files)}] {os.path.basename(image_path)} Count: {count}")

# -------------------------
# Save CSV
# -------------------------
csv_path = os.path.join(run_dir, "counts.csv")
df = pd.DataFrame(outputs)
df.to_csv(csv_path, index=False)
print(f"[INFO] Saved CSV: {csv_path}")

[INFO] Saving outputs to /home/debasish/Documents/YOLOv8/outputs/bacteria_counting/run2
Total images found: 60
[1/60] TileScan 1_Position 1_t00_ch00.jpg Count: 20
[2/60] TileScan 1_Position 1_t01_ch00.jpg Count: 27
[3/60] TileScan 1_Position 1_t02_ch00.jpg Count: 28
[4/60] TileScan 1_Position 1_t03_ch00.jpg Count: 29
[5/60] TileScan 1_Position 1_t04_ch00.jpg Count: 36
[6/60] TileScan 1_Position 1_t05_ch00.jpg Count: 45
[7/60] TileScan 1_Position 1_t06_ch00.jpg Count: 61
[8/60] TileScan 1_Position 1_t07_ch00.jpg Count: 70
[9/60] TileScan 1_Position 1_t08_ch00.jpg Count: 81
[10/60] TileScan 1_Position 1_t09_ch00.jpg Count: 113
[11/60] TileScan 1_Position 1_t10_ch00.jpg Count: 136
[12/60] TileScan 1_Position 1_t11_ch00.jpg Count: 155
[13/60] TileScan 1_Position 2_t00_ch00.jpg Count: 19
[14/60] TileScan 1_Position 2_t01_ch00.jpg Count: 26
[15/60] TileScan 1_Position 2_t02_ch00.jpg Count: 33
[16/60] TileScan 1_Position 2_t03_ch00.jpg Count: 30
[17/60] TileScan 1_Position 2_t04_ch00.jpg Coun